In [1]:
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import os
import re

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
repl_list = ['s/o','d/o','w/o','/','&',',','-']

def clean_data(name):
	name = str(name).lower()
	name = (''.join(i for i in name if ord(i)<128)).strip()
	for repl in repl_list:
		name = name.replace(repl," ")
	if '@' in name:
		pos = name.find('@')
		name = name[:pos].strip()
	name = name.split(" ")
	name = " ".join([each.strip() for each in name])
	return name

def remove_records(merged_data):
	merged_data['delete'] = 0
	merged_data.loc[merged_data['name'].str.find('with') != -1,'delete'] = 1	
	merged_data.loc[merged_data['count_words']>=5,'delete']=1
	merged_data.loc[merged_data['count_words']==0,'delete']=1
	merged_data.loc[merged_data['name'].str.contains(r'\d') == True,'delete']=1
	cleaned_data = merged_data[merged_data.delete==0]
	return cleaned_data

merged_data = pd.concat((male_data,female_data),axis=0)

merged_data['name'] = merged_data['name'].apply(clean_data)
merged_data['count_words'] = merged_data['name'].str.split().apply(len)

cleaned_data = remove_records(merged_data)

indian_cleaned_data = cleaned_data[['name','count_words']].drop_duplicates(subset='name',keep='first')
indian_cleaned_data['label'] = 'indian'

len(indian_cleaned_data)

In [ ]:
from faker import Faker
fake = Faker(‘en_US’)
fake.name()

In [ ]:
non_indian_data.head()

In [ ]:

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers.embeddings import Embedding
from keras.utils import to_categorical
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import Callback
np.random.seed(42)
def char_encoded_representation(data,tokenizer,vocab_size,max_len):
  char_index_sentences = tokenizer.texts_to_sequences(data)
  sequences = [to_categorical(x, num_classes=vocab_size) for x in   char_index_sentences]
  X = sequence.pad_sequences(sequences, maxlen=max_len)
  return X

def build_model(hidden_units,max_len,vocab_size):
  model = Sequential()
  model.add(LSTM(hidden_units,input_shape=(max_len,vocab_size)))
  model.add(Dense(1, activation=’sigmoid’))
  model.compile(loss=’binary_crossentropy’, optimizer=’adam’,   metrics=[‘accuracy’])
  print(model.summary())
  return model
model = build_model(100,max_len,vocab_size)
model.fit(X_train, y_train, epochs=20, batch_size=64,callbacks=myCallback(X_test,y_test))